In [ ]:
import pandas as pd
from clickhouse_driver import Client

CH_HOST = 'localhost'
CH_PORT = 9000
CH_USER = 'default'
CH_PASSWORD = ''
CH_DATABASE = 'default'
CH_TABLE = 'family_cards'

csv_file = 'synthetic_family_cards.csv'
df = pd.read_csv(csv_file, encoding='utf-8-sig')
df.head()


,family_id,region,city,address,family_members_count,children_count,average_income,has_social_support,housing_type,family_category,...,last_update_date,contact_phone,email,has_disabled_member,has_elderly_member,children_under_7,children_7_18,working_members,education_level,health_status
0,FAM-97401,Актобе,South David,sdv-id-yIZDlK,1,0,289952.92,False,квартира,пенсионеры,...,2025-06-17,8 073 683 3476,shelbydavidson@example.com,True,True,0,0,1,среднее специальное,удовлетворительное
1,FAM-11472,Тараз,Mcdonaldhaven,sdv-id-yoBQkv,2,2,163637.31,True,частный дом,многодетная,...,2025-06-17,+7 273 991 90 41,mary40@example.org,True,True,1,2,0,высшее,плохое
2,FAM-11472,Тараз,East Kennethfurt,sdv-id-BiQTfw,5,3,372414.73,False,общежитие,малоимущая,...,2025-06-17,+7 544 381 3405,tmercado@example.net,False,False,1,0,0,высшее,плохое
3,FAM-23362,Нур-Султан,Deckermouth,sdv-id-Gbcdpe,2,1,202393.57,True,квартира,пенсионеры,...,2025-06-17,8 781 142 19 25,elee@example.net,False,True,1,0,0,начальное,плохое
4,FAM-21464,Тараз,Davidberg,sdv-id-iJYYAv,3,0,175574.30,True,квартира,инвалиды,...,2025-03-05,84633471809,davidgreen@example.org,True,False,0,2,0,среднее,удовлетворительное


In [ ]:
df['has_social_support'] = df['has_social_support'].astype(int)
df['has_disabled_member'] = df['has_disabled_member'].astype(int)
df['has_elderly_member'] = df['has_elderly_member'].astype(int)

columns_to_clip = [
    'family_members_count',
    'children_count',
    'children_under_7',
    'children_7_18',
    'working_members'
]

for col in columns_to_clip:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).clip(lower=0).astype('uint8')

df['registration_date'] = pd.to_datetime(df['registration_date']).dt.date
df['last_update_date'] = pd.to_datetime(df['last_update_date']).dt.date


In [ ]:
client = Client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD, database=CH_DATABASE)
client.execute(f"DROP TABLE IF EXISTS {CH_TABLE}")
print(f"🗑 Таблица '{CH_TABLE}' удалена (если существовала).")

🗑 Таблица 'family_cards' удалена (если существовала).


In [ ]:
def create_ch_table():
    client = Client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD, database=CH_DATABASE)
    
    create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {CH_TABLE} (
        family_id String,
        region String,
        city String,
        address String,
        family_members_count UInt8,
        children_count UInt8,
        average_income Float64,
        has_social_support UInt8,
        housing_type String,
        family_category String,
        registration_date Date,
        last_update_date Date,
        contact_phone String,
        email String,
        has_disabled_member UInt8,
        has_elderly_member UInt8,
        children_under_7 UInt8,
        children_7_18 UInt8,
        working_members UInt8,
        education_level String,
        health_status String
    )
    ENGINE = MergeTree()
    ORDER BY tuple()
    """
    
    client.execute(create_table_sql)
    print(f"Таблица '{CH_TABLE}' создана.")


In [ ]:
def upload_to_ch():
    client = Client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD, database=CH_DATABASE)
    data = [tuple(row) for row in df.to_numpy()]
    columns = df.columns.tolist()

    try:
        client.execute(
            f"INSERT INTO {CH_TABLE} ({', '.join(columns)}) VALUES",
            data,
            types_check=True
        )
        print(f"Загружено {len(df)} записей в таблицу '{CH_TABLE}'.")
    except Exception as e:
        print("Ошибка при вставке данных:")
        print(e)


In [ ]:
def connect_to_superset():
    print(f"""
Теперь подключите таблицу в Superset:

1. Перейдите в: Data → Datasets
2. Нажмите: + Dataset
3. Выберите источник ClickHouse
4. База данных: {CH_DATABASE}
5. Таблица: {CH_TABLE}
6. Нажмите Save — и можно строить визуализации!
    """)


In [14]:
create_ch_table()
upload_to_ch()
connect_to_superset()


✅ Таблица 'family_cards' создана.
📥 Загружено 10000 записей в таблицу 'family_cards'.

✅ Теперь подключите таблицу в Superset:

1. Перейдите в: Data → Datasets
2. Нажмите: + Dataset
3. Выберите источник ClickHouse
4. База данных: default
5. Таблица: family_cards
6. Нажмите Save — и можно строить визуализации!
    
